In [3]:
import pandas as pd
import numpy as np
import pickle


import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
import shutil
from d2v_recommender import *
from config import config

from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import roc_auc
import lightgbm
import sklearn

In [ ]:
d2v = D2V_Recommender()
data_dict = d2v.load_data_dict(config.data_dict_path)
print(data_dict["X_train"].shape, data_dict["y_train"].shape, data_dict["X_test"].shape, data_dict["y_test"].shape)
X_train = data_dict["X_train"]
X_test = data_dict["X_test"]
y_train = data_dict["y_train"]
y_test = data_dict["y_test"]

In [ ]:
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [ ]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 1
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

In [ ]:
# # Auto-ML

# shutil.rmtree(config.output_automl_path, ignore_errors=True)
# shutil.rmtree(config.tmp_automl_path, ignore_errors=True)
# automl = AutoSklearnClassifier(
#     time_left_for_this_task=180,
#     per_run_time_limit=60,
#     tmp_folder=config.tmp_automl_path,  # automated
#     output_folder=config.output_automl_path,
#     # metric=roc_auc
# )

# automl.fit(X_train, 
#             y_train,
#             dataset_name='d2v_small')

# ############################################################################
# # Print the final ensemble constructed by auto-sklearn
# # ====================================================

# print(automl.show_models())

In [ ]:
model.score(X_test, y_test)
# automl.cv_results_

In [ ]:
from sklearn.metrics import *

###########################################################################
# Get the Score of the final ensemble
# ===================================

predictions = model.predict(X_test)
proba = model.predict_proba(X_test)

print("Accuracy score:", accuracy_score(y_test, predictions))
print("MCC:", matthews_corrcoef(y_test, predictions))

In [ ]:
1 - y_test.sum()/y_test.shape

In [ ]:
# Add to the recommender
recommender.set_classifier(model)
# predict
recommender.predict(1,133)

In [ ]:
# # TODO
# def automl_binary_classifier(x):
#     return automl.predict_proba(x)[0][0]
# sklearn.metrics.plot_roc_curve(automl_binary_classifier, X_test, y_test)

In [ ]:
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["Ignored","Matched"])
disp.plot()